<a href="https://colab.research.google.com/github/mbhuiyan1/Deep-Learning-Researches/blob/main/image_classification_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential # Sequential API let's you create a model layer by layer. 

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import numpy as np 


**Conecting this notebook with google-drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# For accessing files in google drive.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Bring in the data into this notebook**

In [ ]:
# Our image datastes have different dimensions, so we define our preferred dimension
img_width = 150
img_height = 150

train_data_dir = '/content/drive/MyDrive/ML-image-classification/image_data/training'
validation_data_dir = '/content/drive/MyDrive/ML-image-classification/image_data/validation'
train_samples = 20 
validation_samples = 10
epochs = 10
batch_size = 10   

**Define a simple CNN model**

In [ ]:
model = Sequential() 

model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3))) 
model.add(Activation('relu'))  
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))  
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 activation_55 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_56 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


**Data Augmentation**

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2, # counterclokwise direction
    zoom_range=0.2, # randomly zooming inside the images
    horizontal_flip=True) # Randomly flips horizontally


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 198 images belonging to 2 classes.


In [ ]:

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
print(train_generator.class_indices)

{'Cat': 0, 'Dog': 1}


In [ ]:
imgs, labels = next(train_generator)

In [ ]:
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
image_batch,label_batch = train_generator.next() # The next() function returns the next item in an iterator.
# Now if I run this code it will give me 10 images of dog and cat since batch size =10. If I run the code again it will give me 10 different images from next batch.

print(len(image_batch))
for i in range(0,len(image_batch)):
    image = image_batch[i]
    print(label_batch[i])
    imshow(image)

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 100 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import operator
from pathlib import Path
import logging, os
from google.colab import files
from keras.preprocessing import image


In [ ]:
#predict from test repository
dog_counter = 0 
cat_counter  = 0
image_path = "/content/drive/MyDrive/ML-image-classification/image_data/test/"

images = []
    
# load all images into a list
for im in os.listdir(image_path):
        img = os.path.join(image_path, im)
        img = image.load_img(img, target_size=(img_width, img_height)) # this is a PIL image
        x = image.img_to_array(img) # convert the PIl image to 3d tensor with shape (148, 148, 3)
        x = np.expand_dims(x, axis=0) # # convert the  3d tensor tp 4D tesnor with shape (1, 148, 148, 3)

        images = np.vstack([x]) #all the images in a stack
        classes = model.predict(x)

        print(classes[0])
        plt.axis("off")
        plt.imshow(img)
        plt.show()
        
        if classes[0] <= 0.5:
          print(im + ": " + 'cat\n')
          cat_counter += 1
        else:
          print(im + ": " + 'dog\n' )
          dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)